In [281]:
import pandas as pd
import numpy as np

In [282]:
nba = pd.read_excel("NBA Stats 202122 All Player Statistics in one Page-2.xlsx")

In [283]:
nba.head()

,NBA Stats 2021/22: All Player Statistics in one Page,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
0,RANK,FULL NAME,TEAM,POS,AGE,GP,MPG,MIN%Minutes PercentagePercentage of team minut...,"USG%Usage RateUsage rate, a.k.a., usage percen...",TO%Turnover RateA metric that estimates the nu...,...,RPGReboundsRebounds per game.,TRB%Total Rebound PercentageTotal rebound perc...,APGAssistsAssists per game.,AST%Assist PercentageAssist percentage is an e...,SPGStealsSteals per game.,BPGBlocksBlocks per game.,TOPGTurnoversTurnovers per game.,VIVersatility IndexVersatility index is a metr...,ORTGOffensive RatingIndividual offensive ratin...,DRTGDefensive RatingIndividual defensive ratin...
1,NaN,Nemanja Bjelica,Gol,F,34.06,10,12.1,25.3,12.2,14.7,...,2.3,10.6,1.6,17.2,0.2,0.2,0.5,7.7,121.5,97.4
2,NaN,Jaylen Brown,Bos,G-F,25.6,18,38.2,79.5,26.4,13.5,...,6.7,9.9,3.5,16.5,1.28,0.44,3,8.5,110.2,101.4
3,NaN,Stephen Curry,Gol,G,34.21,16,33.6,70.1,31.5,10.8,...,4.9,8.2,6.2,29.2,1,0.44,2.63,11,117.5,104.4
4,NaN,Malik Fitts,Bos,F,20,6,1.6,3.3,14.3,0,...,0.3,11.8,0,0,0,0,0,0,NaN,NaN


In [284]:
nba = nba.drop(columns=["NBA Stats 2021/22: All Player Statistics in one Page", "Unnamed: 2"])

In [285]:
nba.head()

,Unnamed: 1,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
0,FULL NAME,POS,AGE,GP,MPG,MIN%Minutes PercentagePercentage of team minut...,"USG%Usage RateUsage rate, a.k.a., usage percen...",TO%Turnover RateA metric that estimates the nu...,FTA,FT%,...,RPGReboundsRebounds per game.,TRB%Total Rebound PercentageTotal rebound perc...,APGAssistsAssists per game.,AST%Assist PercentageAssist percentage is an e...,SPGStealsSteals per game.,BPGBlocksBlocks per game.,TOPGTurnoversTurnovers per game.,VIVersatility IndexVersatility index is a metr...,ORTGOffensive RatingIndividual offensive ratin...,DRTGDefensive RatingIndividual defensive ratin...
1,Nemanja Bjelica,F,34.06,10,12.1,25.3,12.2,14.7,7,0.571,...,2.3,10.6,1.6,17.2,0.2,0.2,0.5,7.7,121.5,97.4
2,Jaylen Brown,G-F,25.6,18,38.2,79.5,26.4,13.5,100,0.75,...,6.7,9.9,3.5,16.5,1.28,0.44,3,8.5,110.2,101.4
3,Stephen Curry,G,34.21,16,33.6,70.1,31.5,10.8,101,0.822,...,4.9,8.2,6.2,29.2,1,0.44,2.63,11,117.5,104.4
4,Malik Fitts,F,20,6,1.6,3.3,14.3,0,0,0,...,0.3,11.8,0,0,0,0,0,0,NaN,NaN


In [286]:
row1.columns

Index(['Unnamed: 1', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6',
       'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27',
       'Unnamed: 28'],
      dtype='object')

In [287]:
row1 = nba.loc[nba.index==0]

In [288]:
column = []
for x in row1.columns:
    column.append(row1[x][0])

In [289]:
column

['FULL NAME',
 'POS',
 'AGE',
 'GP',
 'MPG',
 'MIN%Minutes PercentagePercentage of team minutes used by a player while he was on the floor',
 'USG%Usage RateUsage rate, a.k.a., usage percentage is an estimate of the percentage of team plays used by a player while he was on the floor',
 'TO%Turnover RateA metric that estimates the number of turnovers a player commits per 100 possessions',
 'FTA',
 'FT%',
 '2PA',
 '2P%',
 '3PA',
 '3P%',
 'eFG%Effective Shooting PercentageWith eFG%, three-point shots made are worth 50% more than two-point shots made. eFG% Formula=(FGM+ (0.5 x 3PM))/FGA',
 'TS%True Shooting PercentageTrue shooting percentage is a measure of shooting efficiency that takes into account field goals, 3-point field goals, and free throws.',
 'PPGPointsPoints per game.',
 'RPGReboundsRebounds per game.',
 'TRB%Total Rebound PercentageTotal rebound percentage is estimated percentage of available rebounds grabbed by the player while the player is on the court.',
 'APGAssistsAssist

In [290]:
nba.columns = column

In [291]:
nba = nba.drop(index=0)

In [292]:
nba

,FULL NAME,POS,AGE,GP,MPG,MIN%Minutes PercentagePercentage of team minutes used by a player while he was on the floor,"USG%Usage RateUsage rate, a.k.a., usage percentage is an estimate of the percentage of team plays used by a player while he was on the floor",TO%Turnover RateA metric that estimates the number of turnovers a player commits per 100 possessions,FTA,FT%,...,RPGReboundsRebounds per game.,TRB%Total Rebound PercentageTotal rebound percentage is estimated percentage of available rebounds grabbed by the player while the player is on the court.,APGAssistsAssists per game.,AST%Assist PercentageAssist percentage is an estimated percentage of teammate field goals a player assisted while the player is on the court,SPGStealsSteals per game.,BPGBlocksBlocks per game.,TOPGTurnoversTurnovers per game.,"VIVersatility IndexVersatility index is a metric that measures a player’s ability to produce in points, assists, and rebounds. The average player will score around a five on the index, while top players score above 10",ORTGOffensive RatingIndividual offensive rating is the number of points produced by a player per 100 total individual possessions.,DRTGDefensive RatingIndividual defensive rating estimates how many points the player allowed per 100 possessions he individually faced while staying on the court.
1,Nemanja Bjelica,F,34.06,10,12.1,25.3,12.2,14.7,7,0.571,...,2.3,10.6,1.6,17.2,0.2,0.2,0.5,7.7,121.5,97.4
2,Jaylen Brown,G-F,25.6,18,38.2,79.5,26.4,13.5,100,0.75,...,6.7,9.9,3.5,16.5,1.28,0.44,3,8.5,110.2,101.4
3,Stephen Curry,G,34.21,16,33.6,70.1,31.5,10.8,101,0.822,...,4.9,8.2,6.2,29.2,1,0.44,2.63,11,117.5,104.4
4,Malik Fitts,F,20,6,1.6,3.3,14.3,0,0,0,...,0.3,11.8,0,0,0,0,0,0,NaN,NaN
5,Draymond Green,F,32.24,16,30.4,63.2,14.5,27.3,35,0.657,...,6.9,12.8,6.3,27.1,0.94,1.13,2.75,9.5,110.6,96.2
6,Sam Hauser,F,24.47,2,2,4.1,21.8,0,2,1,...,0.5,14.4,0,0,0,0,0,0,NaN,NaN
7,Al Horford,C-F,35.99,17,36.7,76.5,13,10.6,24,0.833,...,9.6,14.7,3.5,14.5,0.88,1.59,1.12,8,128.2,99.3
8,Andre Iguodala,G-F,38.33,3,14.1,29.4,9.6,32.2,3,0.667,...,2.3,9.3,2.3,19.3,0,0.67,1,5.4,94.2,94.4
9,Luke Kornet,F-C,26.87,6,2.1,4.4,7.1,0,0,0,...,0.5,13.2,0,0,0,0,0,0,NaN,NaN
10,Jonathan Kuminga,F,19.65,12,10.8,22.5,27.4,13.5,26,0.769,...,2.2,11.2,0.7,9.5,0.25,0.17,0.92,8.1,106.7,102.7


In [293]:
nba["total_attempts"] = nba["3PA"] + nba["2PA"]

In [294]:
nba["3PM"] = nba["3PA"] * nba["3P%"]
nba["2PM"] = nba["2PA"] * nba["2P%"]
nba["TM"] = nba["3PM"] + nba["2PM"]
nba["3PG"] = nba["3PM"] / nba["GP"]
nba["2PG"] = nba["2PM"] / nba["GP"]
nba["FTM"] = nba["FT%"] * nba["FTA"]
nba["FTMPG"] = nba["FTM"] / nba["GP"]

In [295]:
nba

,FULL NAME,POS,AGE,GP,MPG,MIN%Minutes PercentagePercentage of team minutes used by a player while he was on the floor,"USG%Usage RateUsage rate, a.k.a., usage percentage is an estimate of the percentage of team plays used by a player while he was on the floor",TO%Turnover RateA metric that estimates the number of turnovers a player commits per 100 possessions,FTA,FT%,...,ORTGOffensive RatingIndividual offensive rating is the number of points produced by a player per 100 total individual possessions.,DRTGDefensive RatingIndividual defensive rating estimates how many points the player allowed per 100 possessions he individually faced while staying on the court.,total_attempts,3PM,2PM,TM,3PG,2PG,FTM,FTMPG
1,Nemanja Bjelica,F,34.06,10,12.1,25.3,12.2,14.7,7,0.571,...,121.5,97.4,26,1.998,12.0,13.998,0.1998,1.2,3.997,0.3997
2,Jaylen Brown,G-F,25.6,18,38.2,79.5,26.4,13.5,100,0.75,...,110.2,101.4,303,44.004,103.005,147.009,2.444667,5.7225,75.0,4.166667
3,Stephen Curry,G,34.21,16,33.6,70.1,31.5,10.8,101,0.822,...,117.5,104.4,303,60.04,75.98,136.02,3.7525,4.74875,83.022,5.188875
4,Malik Fitts,F,20,6,1.6,3.3,14.3,0,0,0,...,NaN,NaN,3,1,1.0,2.0,0.166667,0.166667,0,0.0
5,Draymond Green,F,32.24,16,30.4,63.2,14.5,27.3,35,0.657,...,110.6,96.2,102,6.003,48.98,54.983,0.375188,3.06125,22.995,1.437188
6,Sam Hauser,F,24.47,2,2,4.1,21.8,0,2,1,...,NaN,NaN,1,0,0,0,0.0,0.0,2,1.0
7,Al Horford,C-F,35.99,17,36.7,76.5,13,10.6,24,0.833,...,128.2,99.3,150,31.968,43.016,74.984,1.880471,2.530353,19.992,1.176
8,Andre Iguodala,G-F,38.33,3,14.1,29.4,9.6,32.2,3,0.667,...,94.2,94.4,5,0,0.999,0.999,0.0,0.333,2.001,0.667
9,Luke Kornet,F-C,26.87,6,2.1,4.4,7.1,0,0,0,...,NaN,NaN,2,0,1.0,1.0,0.0,0.166667,0,0.0
10,Jonathan Kuminga,F,19.65,12,10.8,22.5,27.4,13.5,26,0.769,...,106.7,102.7,59,3.003,27.002,30.005,0.25025,2.250167,19.994,1.666167


In [296]:
nba.head()

,FULL NAME,POS,AGE,GP,MPG,MIN%Minutes PercentagePercentage of team minutes used by a player while he was on the floor,"USG%Usage RateUsage rate, a.k.a., usage percentage is an estimate of the percentage of team plays used by a player while he was on the floor",TO%Turnover RateA metric that estimates the number of turnovers a player commits per 100 possessions,FTA,FT%,...,ORTGOffensive RatingIndividual offensive rating is the number of points produced by a player per 100 total individual possessions.,DRTGDefensive RatingIndividual defensive rating estimates how many points the player allowed per 100 possessions he individually faced while staying on the court.,total_attempts,3PM,2PM,TM,3PG,2PG,FTM,FTMPG
1,Nemanja Bjelica,F,34.06,10,12.1,25.3,12.2,14.7,7,0.571,...,121.5,97.4,26,1.998,12.0,13.998,0.1998,1.2,3.997,0.3997
2,Jaylen Brown,G-F,25.6,18,38.2,79.5,26.4,13.5,100,0.75,...,110.2,101.4,303,44.004,103.005,147.009,2.444667,5.7225,75.0,4.166667
3,Stephen Curry,G,34.21,16,33.6,70.1,31.5,10.8,101,0.822,...,117.5,104.4,303,60.04,75.98,136.02,3.7525,4.74875,83.022,5.188875
4,Malik Fitts,F,20,6,1.6,3.3,14.3,0,0,0,...,NaN,NaN,3,1,1.0,2.0,0.166667,0.166667,0,0.0
5,Draymond Green,F,32.24,16,30.4,63.2,14.5,27.3,35,0.657,...,110.6,96.2,102,6.003,48.98,54.983,0.375188,3.06125,22.995,1.437188


In [297]:
nba.sort_values("APGAssistsAssists per game.")

,FULL NAME,POS,AGE,GP,MPG,MIN%Minutes PercentagePercentage of team minutes used by a player while he was on the floor,"USG%Usage RateUsage rate, a.k.a., usage percentage is an estimate of the percentage of team plays used by a player while he was on the floor",TO%Turnover RateA metric that estimates the number of turnovers a player commits per 100 possessions,FTA,FT%,...,ORTGOffensive RatingIndividual offensive rating is the number of points produced by a player per 100 total individual possessions.,DRTGDefensive RatingIndividual defensive rating estimates how many points the player allowed per 100 possessions he individually faced while staying on the court.,total_attempts,3PM,2PM,TM,3PG,2PG,FTM,FTMPG
4,Malik Fitts,F,20,6,1.6,3.3,14.3,0,0,0,...,NaN,NaN,3,1,1.0,2.0,0.166667,0.166667,0,0.0
6,Sam Hauser,F,24.47,2,2,4.1,21.8,0,2,1,...,NaN,NaN,1,0,0,0,0.0,0.0,2,1.0
14,Juwan Morgan,F,25.12,5,1.9,4,8.9,0,2,0.5,...,NaN,NaN,1,0,0,0,0.0,0.0,1.0,0.2
9,Luke Kornet,F-C,26.87,6,2.1,4.4,7.1,0,0,0,...,NaN,NaN,2,0,1.0,1.0,0.0,0.166667,0,0.0
15,Aaron Nesmith,G-F,22.62,10,3.7,7.6,14.9,8.3,0,0,...,NaN,NaN,11,1.0,0.999,1.999,0.1,0.0999,0,0.0
21,Nik Stauskas,G,28.64,8,1.9,4,29,0,4,1,...,NaN,NaN,8,1.998,0,1.998,0.24975,0.0,4,0.5
13,Moses Moody,G,20,9,10.4,21.7,15.2,9.2,6,0.667,...,121.4,110,27,6.994,7.0,13.994,0.777111,0.777778,4.002,0.444667
25,Juan Toscano-Anderson,F,29.14,10,4.1,8.5,17.4,36.8,3,0.333,...,76.4,107.1,9,2.0,2.0,4.0,0.2,0.2,0.999,0.0999
11,Damion Lee,G-F,29.6,12,9.7,20.1,14,16.1,3,0.667,...,98.9,105.1,30,3.996,9.0,12.996,0.333,0.75,2.001,0.16675
23,Daniel Theis,F-C,30.15,14,12.8,26.7,15.7,16,8,0.75,...,116.8,94.8,49,2.004,27.01,29.014,0.143143,1.929286,6.0,0.428571


In [298]:
nba[["APGAssistsAssists per game.", "RPGReboundsRebounds per game."]]

,APGAssistsAssists per game.,RPGReboundsRebounds per game.
1,1.6,2.3
2,3.5,6.7
3,6.2,4.9
4,0,0.3
5,6.3,6.9
6,0,0.5
7,3.5,9.6
8,2.3,2.3
9,0,0.5
10,0.7,2.2


In [299]:
nba_stats = nba[["FULL NAME", "APGAssistsAssists per game.", "RPGReboundsRebounds per game.", "PPGPointsPoints per game.", "3P%", "BPGBlocksBlocks per game.", "TOPGTurnoversTurnovers per game.", "total_attempts", "3P%", "3PA", "2PA", "3PG", "2PG", "FTMPG", "SPGStealsSteals per game."]]








In [300]:
score = []
for stat, row in nba_stats.iterrows():
    scores = round(row["APGAssistsAssists per game."]) * 1.5 + round(row["RPGReboundsRebounds per game."]) * 1.25 + round(row["3PG"] * 3) * 1.5 + round(row["2PG"] * 2) + round(row["FTMPG"]) + round(row["RPGReboundsRebounds per game."]) * 1.25 - round(row["TOPGTurnoversTurnovers per game."]) * 0.5 + round(row["BPGBlocksBlocks per game."]) * 1.5 + round(row["SPGStealsSteals per game."] * 2)
    score.append(scores)

In [301]:
nba_stats["scores"] = score

/Users/evanbruno/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [302]:
nba_stats.sort_values("scores")

,FULL NAME,APGAssistsAssists per game.,RPGReboundsRebounds per game.,PPGPointsPoints per game.,3P%,BPGBlocksBlocks per game.,TOPGTurnoversTurnovers per game.,total_attempts,3P%,3PA,2PA,3PG,2PG,FTMPG,SPGStealsSteals per game.,scores
4,Malik Fitts,0,0.3,0.8,1,0,0,3,1,1,2,0.166667,0.166667,0.0,0,0.0
14,Juwan Morgan,0,0.4,0.2,0,0,0,1,0,1,0,0.0,0.0,0.2,0,0.0
9,Luke Kornet,0,0.5,0.3,0,0,0,2,0,0,2,0.0,0.166667,0.0,0,0.0
6,Sam Hauser,0,0.5,1,0,0,0,1,0,1,0,0.0,0.0,1.0,0,1.0
21,Nik Stauskas,0.4,0.1,1.3,0.333,0,0,8,0.333,6,2,0.24975,0.0,0.5,0,1.5
15,Aaron Nesmith,0.1,1.1,0.5,0.125,0.4,0.1,11,0.125,8,3,0.1,0.0999,0.0,0.1,2.5
25,Juan Toscano-Anderson,0.5,0.8,1.1,0.5,0.1,0.6,9,0.5,4,5,0.2,0.2,0.0999,0.2,3.5
13,Moses Moody,0.4,0.9,4.3,0.538,0.22,0.33,27,0.538,13,14,0.777111,0.777778,0.444667,0.33,8.5
11,Damion Lee,0.6,2.2,2.7,0.333,0,0.5,30,0.333,12,18,0.333,0.75,0.16675,0.08,10.0
8,Andre Iguodala,2.3,2.3,1.3,0,0.67,1,5,0,2,3,0.0,0.333,0.667,0,11.0


In [303]:
nba_stats

,FULL NAME,APGAssistsAssists per game.,RPGReboundsRebounds per game.,PPGPointsPoints per game.,3P%,BPGBlocksBlocks per game.,TOPGTurnoversTurnovers per game.,total_attempts,3P%,3PA,2PA,3PG,2PG,FTMPG,SPGStealsSteals per game.,scores
1,Nemanja Bjelica,1.6,2.3,3.4,0.333,0.2,0.5,26,0.333,6,20,0.1998,1.2,0.3997,0.2,11.5
2,Jaylen Brown,3.5,6.7,22.9,0.386,0.44,3,303,0.386,114,189,2.444667,5.7225,4.166667,1.28,50.5
3,Stephen Curry,6.2,4.9,25.9,0.38,0.44,2.63,303,0.38,158,145,3.7525,4.74875,5.188875,1,52.5
4,Malik Fitts,0,0.3,0.8,1,0,0,3,1,1,2,0.166667,0.166667,0.0,0,0.0
5,Draymond Green,6.3,6.9,8.7,0.261,1.13,2.75,102,0.261,23,79,0.375188,3.06125,1.437188,0.94,37.0
6,Sam Hauser,0,0.5,1,0,0,0,1,0,1,0,0.0,0.0,1.0,0,1.0
7,Al Horford,3.5,9.6,11.9,0.432,1.59,1.12,150,0.432,74,76,1.880471,2.530353,1.176,0.88,50.5
8,Andre Iguodala,2.3,2.3,1.3,0,0.67,1,5,0,2,3,0.0,0.333,0.667,0,11.0
9,Luke Kornet,0,0.5,0.3,0,0,0,2,0,0,2,0.0,0.166667,0.0,0,0.0
10,Jonathan Kuminga,0.7,2.2,6.9,0.231,0.17,0.92,59,0.231,13,46,0.25025,2.250167,1.666167,0.25,14.5


In [304]:
nba_stats.rename(columns={"FULL NAME": "Name"}, inplace=True)

/Users/evanbruno/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [305]:
DKG_salary = pd.read_csv("DKSalaries.csv")

In [306]:
DKG_salary

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
0,PG,Stephen Curry (22935509),Stephen Curry,22935509,CPT,15900,BOS@GSW 06/05/2022 08:00PM ET,GSW,45.71
1,SF/PF,Jayson Tatum (22935510),Jayson Tatum,22935510,CPT,15600,BOS@GSW 06/05/2022 08:00PM ET,BOS,47.50
2,SG/SF,Jaylen Brown (22935511),Jaylen Brown,22935511,CPT,14400,BOS@GSW 06/05/2022 08:00PM ET,BOS,39.54
3,PG,Marcus Smart (22935512),Marcus Smart,22935512,CPT,12000,BOS@GSW 06/05/2022 08:00PM ET,BOS,30.09
4,PF/C,Al Horford (22935513),Al Horford,22935513,CPT,11400,BOS@GSW 06/05/2022 08:00PM ET,BOS,30.47
...,...,...,...,...,...,...,...,...,...
85,SF,Malik Fitts (22935594),Malik Fitts,22935594,UTIL,1000,BOS@GSW 06/05/2022 08:00PM ET,BOS,2.82
86,PF,Al-Farouq Aminu (22935595),Al-Farouq Aminu,22935595,UTIL,1000,BOS@GSW 06/05/2022 08:00PM ET,BOS,0.00
87,SF,Justin Jackson (22935596),Justin Jackson,22935596,UTIL,1000,BOS@GSW 06/05/2022 08:00PM ET,BOS,3.96
88,SF,Axel Toupane (22935597),Axel Toupane,22935597,UTIL,1000,BOS@GSW 06/05/2022 08:00PM ET,GSW,0.00


In [307]:
UTIL = DKG_salary.loc[DKG_salary["Roster Position"] == "UTIL",:]

In [308]:
CPT = DKG_salary.loc[DKG_salary["Roster Position"] == "CPT"]

In [309]:
new_names = nba_stats["Name"].tolist()

In [310]:
new_names[27] = "Robert Williams"

In [311]:
new_names

['Nemanja Bjelica',
 'Jaylen Brown',
 'Stephen Curry',
 'Malik Fitts',
 'Draymond Green',
 'Sam Hauser',
 'Al Horford',
 'Andre Iguodala',
 'Luke Kornet',
 'Jonathan Kuminga',
 'Damion Lee',
 'Kevon Looney',
 'Moses Moody',
 'Juwan Morgan',
 'Aaron Nesmith',
 'Gary Payton II',
 'Jordan Poole',
 'Otto Porter Jr.',
 'Payton Pritchard',
 'Marcus Smart',
 'Nik Stauskas',
 'Jayson Tatum',
 'Daniel Theis',
 'Klay Thompson',
 'Juan Toscano-Anderson',
 'Derrick White',
 'Andrew Wiggins',
 'Robert Williams',
 'Grant Williams']

In [312]:
nba_stats["Name"] = new_names

/Users/evanbruno/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [313]:
s = []
for x in nba_stats["Name"]:

    for row in UTIL.iterrows():
        if x == row[1]["Name"]:
            s.append(row[1]["Salary"])


    
        

In [314]:
nba_stats["UTIL_salary"] = s

/Users/evanbruno/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [315]:
nba_stats["CPT_Salary"] = nba_stats["UTIL_salary"] * 1.5

/Users/evanbruno/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [316]:
nba_stats

,Name,APGAssistsAssists per game.,RPGReboundsRebounds per game.,PPGPointsPoints per game.,3P%,BPGBlocksBlocks per game.,TOPGTurnoversTurnovers per game.,total_attempts,3P%,3PA,2PA,3PG,2PG,FTMPG,SPGStealsSteals per game.,scores,UTIL_salary,CPT_Salary
1,Nemanja Bjelica,1.6,2.3,3.4,0.333,0.2,0.5,26,0.333,6,20,0.1998,1.2,0.3997,0.2,11.5,1600,2400.0
2,Jaylen Brown,3.5,6.7,22.9,0.386,0.44,3,303,0.386,114,189,2.444667,5.7225,4.166667,1.28,50.5,9600,14400.0
3,Stephen Curry,6.2,4.9,25.9,0.38,0.44,2.63,303,0.38,158,145,3.7525,4.74875,5.188875,1,52.5,10600,15900.0
4,Malik Fitts,0,0.3,0.8,1,0,0,3,1,1,2,0.166667,0.166667,0.0,0,0.0,1000,1500.0
5,Draymond Green,6.3,6.9,8.7,0.261,1.13,2.75,102,0.261,23,79,0.375188,3.06125,1.437188,0.94,37.0,7000,10500.0
6,Sam Hauser,0,0.5,1,0,0,0,1,0,1,0,0.0,0.0,1.0,0,1.0,1000,1500.0
7,Al Horford,3.5,9.6,11.9,0.432,1.59,1.12,150,0.432,74,76,1.880471,2.530353,1.176,0.88,50.5,7600,11400.0
8,Andre Iguodala,2.3,2.3,1.3,0,0.67,1,5,0,2,3,0.0,0.333,0.667,0,11.0,1400,2100.0
9,Luke Kornet,0,0.5,0.3,0,0,0,2,0,0,2,0.0,0.166667,0.0,0,0.0,1000,1500.0
10,Jonathan Kuminga,0.7,2.2,6.9,0.231,0.17,0.92,59,0.231,13,46,0.25025,2.250167,1.666167,0.25,14.5,2200,3300.0


In [279]:
for x in range(1, len(nba_stats)):
    combo = nba_stats.drop(index = x)
    
    comb_array = np.array(np.meshgrid(combo["scores"].tolist(), combo["scores"].tolist(), combo["scores"].tolist(), combo["scores"].tolist())).T.reshape(-1, 4)

In [460]:
combo = nba_stats.sort_values("scores", ascending=False).copy()
combo.head()

,Name,APGAssistsAssists per game.,RPGReboundsRebounds per game.,PPGPointsPoints per game.,3P%,BPGBlocksBlocks per game.,TOPGTurnoversTurnovers per game.,total_attempts,3P%,3PA,2PA,3PG,2PG,FTMPG,SPGStealsSteals per game.,scores,UTIL_salary,CPT_Salary
22,Jayson Tatum,5.9,6.7,27,0.375,0.94,4.28,352,0.375,152,200,3.166667,5.555556,6.386333,1.22,60.0,10400,15600.0
3,Stephen Curry,6.2,4.9,25.9,0.38,0.44,2.63,303,0.38,158,145,3.7525,4.74875,5.188875,1,52.5,10600,15900.0
2,Jaylen Brown,3.5,6.7,22.9,0.386,0.44,3,303,0.386,114,189,2.444667,5.7225,4.166667,1.28,50.5,9600,14400.0
7,Al Horford,3.5,9.6,11.9,0.432,1.59,1.12,150,0.432,74,76,1.880471,2.530353,1.176,0.88,50.5,7600,11400.0
27,Andrew Wiggins,1.7,7,15.8,0.353,0.88,1.19,204,0.353,68,136,1.50025,4.624,2.06375,0.81,42.0,7200,10800.0


In [395]:
combo.drop(index=22)

,Name,APGAssistsAssists per game.,RPGReboundsRebounds per game.,PPGPointsPoints per game.,3P%,BPGBlocksBlocks per game.,TOPGTurnoversTurnovers per game.,total_attempts,3P%,3PA,2PA,3PG,2PG,FTMPG,SPGStealsSteals per game.,scores,UTIL_salary,CPT_Salary
3,Stephen Curry,6.2,4.9,25.9,0.38,0.44,2.63,303,0.38,158,145,3.7525,4.74875,5.188875,1,52.5,10600,15900.0
2,Jaylen Brown,3.5,6.7,22.9,0.386,0.44,3,303,0.386,114,189,2.444667,5.7225,4.166667,1.28,50.5,9600,14400.0
7,Al Horford,3.5,9.6,11.9,0.432,1.59,1.12,150,0.432,74,76,1.880471,2.530353,1.176,0.88,50.5,7600,11400.0
27,Andrew Wiggins,1.7,7,15.8,0.353,0.88,1.19,204,0.353,68,136,1.50025,4.624,2.06375,0.81,42.0,7200,10800.0
24,Klay Thompson,2.4,4.3,19.8,0.399,0.75,1.38,266,0.399,143,123,3.566063,4.059,1.0,1,41.5,7400,11100.0
20,Marcus Smart,6.2,4.5,15.5,0.33,0.33,2.2,200,0.33,103,97,2.266,3.000533,2.733333,1.13,39.5,8000,12000.0
5,Draymond Green,6.3,6.9,8.7,0.261,1.13,2.75,102,0.261,23,79,0.375188,3.06125,1.437188,0.94,37.0,7000,10500.0
17,Jordan Poole,4.5,3.2,18.4,0.393,0.44,2.56,192,0.393,89,103,2.186063,4.184375,3.43875,0.94,35.5,6000,9000.0
12,Kevon Looney,2,7.7,6.1,0,0.31,0.56,66,0,0,66,0.0,2.751375,0.563,0.31,30.5,6800,10200.0
28,Robert Williams,0.7,5.5,7.8,0,1.91,0.55,52,0,1,51,0.0,2.999727,1.818,0.55,28.0,6400,9600.0


In [ ]:
total_lists = []
temp_chosen = []
for idx, row in combo.iterrows():
    temp_chosen = []
    total_points = 0
    comb = nba_stats.sort_values("scores", ascending=False).copy()
    remaining = 50000 - row["CPT_Salary"]
    total_points = row["scores"]
    cpt_name = row["Name"]
    temp_chosen.append({row["Name"] : row["scores"] * 1.5})
    comb = comb.drop(index = idx)
    for x in range(5):
        

In [481]:
total_lists = []
temp_chosen = []
for idx, row in combo.iterrows():
    temp_chosen = []
    total_points = 0
    comb = nba_stats.sort_values("scores", ascending=False).copy()
    remaining = 50000 - row["CPT_Salary"]
    total_points = row["scores"]
    cpt_name = row["Name"]
    temp_chosen.append({row["Name"] : row["scores"] * 1.5})
    comb = comb.drop(index = idx)
    for x in range(5):
        for idx, row in comb.iterrows():
            if row["UTIL_salary"] < remaining:
                temp_chosen.append({row["Name"] : row["scores"]})
                remaining = remaining - row["UTIL_salary"]
                index = idx
                total_points = total_points + row["scores"]
                break
            else: 
                index = idx
        comb = comb.drop(index = index)
    temp_chosen.append({"Total Score" : total_points})
    total_lists.append({cpt_name : temp_chosen})

In [495]:
combos = nba_stats.sort_values("scores", ascending=False).copy()
combos.head()

,Name,APGAssistsAssists per game.,RPGReboundsRebounds per game.,PPGPointsPoints per game.,3P%,BPGBlocksBlocks per game.,TOPGTurnoversTurnovers per game.,total_attempts,3P%,3PA,2PA,3PG,2PG,FTMPG,SPGStealsSteals per game.,scores,UTIL_salary,CPT_Salary
22,Jayson Tatum,5.9,6.7,27,0.375,0.94,4.28,352,0.375,152,200,3.166667,5.555556,6.386333,1.22,60.0,10400,15600.0
3,Stephen Curry,6.2,4.9,25.9,0.38,0.44,2.63,303,0.38,158,145,3.7525,4.74875,5.188875,1,52.5,10600,15900.0
2,Jaylen Brown,3.5,6.7,22.9,0.386,0.44,3,303,0.386,114,189,2.444667,5.7225,4.166667,1.28,50.5,9600,14400.0
7,Al Horford,3.5,9.6,11.9,0.432,1.59,1.12,150,0.432,74,76,1.880471,2.530353,1.176,0.88,50.5,7600,11400.0
27,Andrew Wiggins,1.7,7,15.8,0.353,0.88,1.19,204,0.353,68,136,1.50025,4.624,2.06375,0.81,42.0,7200,10800.0


In [498]:
total_lists = []
temp_chosen = []
for idx, row in combos.iterrows():
    temp_chosen = []
    total_points = 0
    comb = nba_stats.sort_values("scores", ascending=False).copy()
    remaining = 50000 - row["CPT_Salary"]
    total_points = row["scores"]
    cpt_name = row["Name"]
    temp_chosen.append({row["Name"] : row["scores"] * 1.5})
    
    legit = ()
    max_score = 0
    max_salary = 50000 - row["CPT_Salary"]
    comb = comb.drop(index = idx)
    for combo in combinations(comb["Name"], 5):  # 2 for pairs, 3 for triplets, etc
        total_score = 0
        total_salary = 0
        for x in combo:
            rowP = nba_stats.loc[nba_stats["Name"] == x]
            total_score = total_score + rowP["scores"].tolist()[0]
            total_salary = total_salary + rowP["UTIL_salary"].tolist()[0]
        if total_salary < max_salary:
            if total_score > max_score:
                print(total_score)
                max_score = total_score
                legit = combo
    total_score = total_score + total_points
    temp_chosen.append({total_score : [legit]})

191.5
200.0
202.5
205.0
199.0
207.5
210.0
202.0
209.5
214.5
219.0
214.5
219.0
225.0
215.0
219.0
225.5
208.5
211.5
225.5
223.5
225.5
229.0
229.5
234.0
224.5
225.5
229.0
231.0
234.0
234.0
240.5
241.5
246.5
255.5
239.5
242.0
244.5
249.0
255.5
255.5
255.5
255.5
255.5
255.5
255.5
255.5
255.5
255.5
255.5
255.5
255.5


In [499]:
temp_chosen

[{'Malik Fitts': 0.0},
 {5.0: [('Jayson Tatum',
    'Stephen Curry',
    'Jaylen Brown',
    'Al Horford',
    'Andrew Wiggins')]}]

In [437]:
import itertools

In [456]:
from itertools import combinations
legit = []
for combo in combinations(nba_stats["UTIL_salary"], 4):  # 2 for pairs, 3 for triplets, etc
    print(combo[0] + combo[1] + combo[2] + combo[3] + 15900)
    if (combo[0] + combo[1] + combo[2] + combo[3] + 15900 < 50000) & (combo[0] + combo[1] + combo[2] + combo[3] + 15900 > 45000):
        legit.append({"Combo" : combo})

38700
44700
38700
45300
39100
38700
39900
38700
44500
38900
38700
38700
39500
43700
42300
41300
45700
38700
48100
39700
45100
38700
43100
44900
44100
41900
35100
29100
35700
29500
29100
30300
29100
34900
29300
29100
29100
29900
34100
32700
31700
36100
29100
38500
30100
35500
29100
33500
35300
34500
32300
35100
41700
35500
35100
36300
35100
40900
35300
35100
35100
35900
40100
38700
37700
42100
35100
44500
36100
41500
35100
39500
41300
40500
38300
35700
29500
29100
30300
29100
34900
29300
29100
29100
29900
34100
32700
31700
36100
29100
38500
30100
35500
29100
33500
35300
34500
32300
36100
35700
36900
35700
41500
35900
35700
35700
36500
40700
39300
38300
42700
35700
45100
36700
42100
35700
40100
41900
41100
38900
29500
30700
29500
35300
29700
29500
29500
30300
34500
33100
32100
36500
29500
38900
30500
35900
29500
33900
35700
34900
32700
30300
29100
34900
29300
29100
29100
29900
34100
32700
31700
36100
29100
38500
30100
35500
29100
33500
35300
34500
32300
30300
36100
30500
30300
30300
3110

25500
24100
23100
27500
20500
29900
21500
26900
20500
24900
26700
25900
23700
26300
24900
23900
28300
21300
30700
22300
27700
21300
25700
27500
26700
24500
29100
28100
32500
25500
34900
26500
31900
25500
29900
31700
30900
28700
26700
31100
24100
33500
25100
30500
24100
28500
30300
29500
27300
30100
23100
32500
24100
29500
23100
27500
29300
28500
26300
27500
36900
28500
33900
27500
31900
33700
32900
30700
29900
21500
26900
20500
24900
26700
25900
23700
30900
36300
29900
34300
36100
35300
33100
27900
21500
25900
27700
26900
24700
26900
31300
33100
32300
30100
24900
26700
25900
23700
31100
30300
28100
32100
29900
29100
27500
28700
27500
33300
27700
27500
27500
28300
32500
31100
30100
34500
27500
36900
28500
33900
27500
31900
33700
32900
30700
28300
27100
32900
27300
27100
27100
27900
32100
30700
29700
34100
27100
36500
28100
33500
27100
31500
33300
32500
30300
28300
34100
28500
28300
28300
29100
33300
31900
30900
35300
28300
37700
29300
34700
28300
32700
34500
33700
31500
32900
27300
2710

32300
30100
24900
26700
25900
23700
31100
30300
28100
32100
29900
29100
29900
28900
33300
26300
35700
27300
32700
26300
30700
32500
31700
29500
27500
31900
24900
34300
25900
31300
24900
29300
31100
30300
28100
30900
23900
33300
24900
30300
23900
28300
30100
29300
27100
28300
37700
29300
34700
28300
32700
34500
33700
31500
30700
22300
27700
21300
25700
27500
26700
24500
31700
37100
30700
35100
36900
36100
33900
28700
22300
26700
28500
27700
25500
27700
32100
33900
33100
30900
25700
27500
26700
24500
31900
31100
28900
32900
30700
29900
31700
36100
29100
38500
30100
35500
29100
33500
35300
34500
32300
35100
28100
37500
29100
34500
28100
32500
34300
33500
31300
32500
41900
33500
38900
32500
36900
38700
37900
35700
34900
26500
31900
25500
29900
31700
30900
28700
35900
41300
34900
39300
41100
40300
38100
32900
26500
30900
32700
31900
29700
31900
36300
38100
37300
35100
29900
31700
30900
28700
36100
35300
33100
37100
34900
34100
33700
26700
36100
27700
33100
26700
31100
32900
32100
29900
3110

34700
33900
31700
38900
44300
37900
42300
44100
43300
41100
35900
29500
33900
35700
34900
32700
34900
39300
41100
40300
38100
32900
34700
33900
31700
39100
38300
36100
40100
37900
37100
35500
36700
35500
41300
35700
35500
35500
36300
40500
39100
38100
42500
35500
44900
36500
41900
35500
39900
41700
40900
38700
36300
35100
40900
35300
35100
35100
35900
40100
38700
37700
42100
35100
44500
36100
41500
35100
39500
41300
40500
38300
36300
42100
36500
36300
36300
37100
41300
39900
38900
43300
36300
45700
37300
42700
36300
40700
42500
41700
39500
40900
35300
35100
35100
35900
40100
38700
37700
42100
35100
44500
36100
41500
35100
39500
41300
40500
38300
41100
40900
40900
41700
45900
44500
43500
47900
40900
50300
41900
47300
40900
45300
47100
46300
44100
35300
35300
36100
40300
38900
37900
42300
35300
44700
36300
41700
35300
39700
41500
40700
38500
35100
35900
40100
38700
37700
42100
35100
44500
36100
41500
35100
39500
41300
40500
38300
35900
40100
38700
37700
42100
35100
44500
36100
41500
3510

30300
34500
33100
32100
36500
29500
38900
30500
35900
29500
33900
35700
34900
32700
30700
36500
30900
30700
30700
31500
35700
34300
33300
37700
30700
40100
31700
37100
30700
35100
36900
36100
33900
35300
29700
29500
29500
30300
34500
33100
32100
36500
29500
38900
30500
35900
29500
33900
35700
34900
32700
35500
35300
35300
36100
40300
38900
37900
42300
35300
44700
36300
41700
35300
39700
41500
40700
38500
29700
29700
30500
34700
33300
32300
36700
29700
39100
30700
36100
29700
34100
35900
35100
32900
29500
30300
34500
33100
32100
36500
29500
38900
30500
35900
29500
33900
35700
34900
32700
30300
34500
33100
32100
36500
29500
38900
30500
35900
29500
33900
35700
34900
32700
35300
33900
32900
37300
30300
39700
31300
36700
30300
34700
36500
35700
33500
38100
37100
41500
34500
43900
35500
40900
34500
38900
40700
39900
37700
35700
40100
33100
42500
34100
39500
33100
37500
39300
38500
36300
39100
32100
41500
33100
38500
32100
36500
38300
37500
35300
36500
45900
37500
42900
36500
40900
42700
4190

...

39700
32700
42100
33700
39100
32700
37100
38900
38100
35900
33900
38300
31300
40700
32300
37700
31300
35700
37500
36700
34500
37300
30300
39700
31300
36700
30300
34700
36500
35700
33500
34700
44100
35700
41100
34700
39100
40900
40100
37900
37100
28700
34100
27700
32100
33900
33100
30900
38100
43500
37100
41500
43300
42500
40300
35100
28700
33100
34900
34100
31900
34100
38500
40300
39500
37300
32100
33900
33100
30900
38300
37500
35300
39300
37100
36300
32500
32300
32300
33100
37300
35900
34900
39300
32300
41700
33300
38700
32300
36700
38500
37700
35500
26700
26700
27500
31700
30300
29300
33700
26700
36100
27700
33100
26700
31100
32900
32100
29900
26500
27300
31500
30100
29100
33500
26500
35900
27500
32900
26500
30900
32700
31900
29700
27300
31500
30100
29100
33500
26500
35900
27500
32900
26500
30900
32700
31900
29700
32300
30900
29900
34300
27300
36700
28300
33700
27300
31700
33500
32700
30500
35100
34100
38500
31500
40900
32500
37900
31500
35900
37700
36900
34700
32700
37100
30100
3950

23900
28300
30100
29300
27100
29900
22900
32300
23900
29300
22900
27300
29100
28300
26100
27300
36700
28300
33700
27300
31700
33500
32700
30500
29700
21300
26700
20300
24700
26500
25700
23500
30700
36100
29700
34100
35900
35100
32900
27700
21300
25700
27500
26700
24500
26700
31100
32900
32100
29900
24700
26500
25700
23500
30900
30100
27900
31900
29700
28900
26300
26300
27100
31300
29900
28900
33300
26300
35700
27300
32700
26300
30700
32500
31700
29500
26100
26900
31100
29700
28700
33100
26100
35500
27100
32500
26100
30500
32300
31500
29300
26900
31100
29700
28700
33100
26100
35500
27100
32500
26100
30500
32300
31500
29300
31900
30500
29500
33900
26900
36300
27900
33300
26900
31300
33100
32300
30100
34700
33700
38100
31100
40500
32100
37500
31100
35500
37300
36500
34300
32300
36700
29700
39100
30700
36100
29700
34100
35900
35100
32900
35700
28700
38100
29700
35100
28700
33100
34900
34100
31900
33100
42500
34100
39500
33100
37500
39300
38500
36300
35500
27100
32500
26100
30500
32300
3150

28100
33500
27100
31500
33300
32500
30300
26900
27700
31900
30500
29500
33900
26900
36300
27900
33300
26900
31300
33100
32300
30100
27700
31900
30500
29500
33900
26900
36300
27900
33300
26900
31300
33100
32300
30100
32700
31300
30300
34700
27700
37100
28700
34100
27700
32100
33900
33100
30900
35500
34500
38900
31900
41300
32900
38300
31900
36300
38100
37300
35100
33100
37500
30500
39900
31500
36900
30500
34900
36700
35900
33700
36500
29500
38900
30500
35900
29500
33900
35700
34900
32700
33900
43300
34900
40300
33900
38300
40100
39300
37100
36300
27900
33300
26900
31300
33100
32300
30100
37300
42700
36300
40700
42500
41700
39500
34300
27900
32300
34100
33300
31100
33300
37700
39500
38700
36500
31300
33100
32300
30100
37500
36700
34500
38500
36300
35500
21300
22100
26300
24900
23900
28300
21300
30700
22300
27700
21300
25700
27500
26700
24500
22100
26300
24900
23900
28300
21300
30700
22300
27700
21300
25700
27500
26700
24500
27100
25700
24700
29100
22100
31500
23100
28500
22100
26500
2830

28900
36100
41500
35100
39500
41300
40500
38300
33100
26700
31100
32900
32100
29900
32100
36500
38300
37500
35300
30100
31900
31100
28900
36300
35500
33300
37300
35100
34300
31500
30100
29100
33500
26500
35900
27500
32900
26500
30900
32700
31900
29700
34300
33300
37700
30700
40100
31700
37100
30700
35100
36900
36100
33900
31900
36300
29300
38700
30300
35700
29300
33700
35500
34700
32500
35300
28300
37700
29300
34700
28300
32700
34500
33700
31500
32700
42100
33700
39100
32700
37100
38900
38100
35900
35100
26700
32100
25700
30100
31900
31100
28900
36100
41500
35100
39500
41300
40500
38300
33100
26700
31100
32900
32100
29900
32100
36500
38300
37500
35300
30100
31900
31100
28900
36300
35500
33300
37300
35100
34300
35100
34100
38500
31500
40900
32500
37900
31500
35900
37700
36900
34700
32700
37100
30100
39500
31100
36500
30100
34500
36300
35500
33300
36100
29100
38500
30100
35500
29100
33500
35300
34500
32300
33500
42900
34500
39900
33500
37900
39700
38900
36700
35900
27500
32900
26500
3090

29300
28300
32700
25700
35100
26700
32100
25700
30100
31900
31100
28900
26900
31300
24300
33700
25300
30700
24300
28700
30500
29700
27500
30300
23300
32700
24300
29700
23300
27700
29500
28700
26500
27700
37100
28700
34100
27700
32100
33900
33100
30900
30100
21700
27100
20700
25100
26900
26100
23900
31100
36500
30100
34500
36300
35500
33300
28100
21700
26100
27900
27100
24900
27100
31500
33300
32500
30300
25100
26900
26100
23900
31300
30500
28300
32300
30100
29300
31100
35500
28500
37900
29500
34900
28500
32900
34700
33900
31700
34500
27500
36900
28500
33900
27500
31900
33700
32900
30700
31900
41300
32900
38300
31900
36300
38100
37300
35100
34300
25900
31300
24900
29300
31100
30300
28100
35300
40700
34300
38700
40500
39700
37500
32300
25900
30300
32100
31300
29100
31300
35700
37500
36700
34500
29300
31100
30300
28100
35500
34700
32500
36500
34300
33500
33100
26100
35500
27100
32500
26100
30500
32300
31500
29300
30500
39900
31500
36900
30500
34900
36700
35900
33700
32900
24500
29900
2350

In [492]:
legit = ()
max_score = 0
max_salary = 42000
for combo in combinations(nba_stats["Name"], 5):  # 2 for pairs, 3 for triplets, etc
    total_score = 0
    total_salary = 0
    for x in combo:
        rowP = nba_stats.loc[nba_stats["Name"] == x]
        total_score = total_score + rowP["scores"].tolist()[0]
        total_salary = total_salary + rowP["UTIL_salary"].tolist()[0]
    if total_salary < max_salary:
        if total_score > max_score:
            print(total_score)
            max_score = total_score
            legit = combo

151.5
165.0
174.5
202.0
211.5
225.0
226.0
228.0
234.0
240.0
240.5


In [493]:
legit

('Stephen Curry',
 'Al Horford',
 'Jordan Poole',
 'Jayson Tatum',
 'Andrew Wiggins')

In [409]:
combo = nba_stats.sort_values("scores", ascending=False).copy()

In [479]:
combo = nba_stats.sort_values("scores", ascending=False).copy()
c = 8000
t = 1.5 * 30
list_chosen = []
remaining = 50000 - c
for x in range(4):
    temp_chosen = []
    for idx, row in comb.iterrows():
        if row["UTIL_salary"] < remaining:
            temp_chosen.append({row["Name"] : row["scores"]})
            remaining = remaining - row["UTIL_salary"]
            index = idx
            break
    comb = comb.drop(index = index)
    list_chosen.append(temp_chosen)
        

In [480]:
list_chosen

[[{'Klay Thompson': 41.5}],
 [{'Marcus Smart': 39.5}],
 [{'Draymond Green': 37.0}],
 [{'Jordan Poole': 35.5}]]

In [327]:
nba_stats.sort_values("scores", ascending=False)

,Name,APGAssistsAssists per game.,RPGReboundsRebounds per game.,PPGPointsPoints per game.,3P%,BPGBlocksBlocks per game.,TOPGTurnoversTurnovers per game.,total_attempts,3P%,3PA,2PA,3PG,2PG,FTMPG,SPGStealsSteals per game.,scores,UTIL_salary,CPT_Salary
22,Jayson Tatum,5.9,6.7,27,0.375,0.94,4.28,352,0.375,152,200,3.166667,5.555556,6.386333,1.22,60.0,10400,15600.0
3,Stephen Curry,6.2,4.9,25.9,0.38,0.44,2.63,303,0.38,158,145,3.7525,4.74875,5.188875,1,52.5,10600,15900.0
2,Jaylen Brown,3.5,6.7,22.9,0.386,0.44,3,303,0.386,114,189,2.444667,5.7225,4.166667,1.28,50.5,9600,14400.0
7,Al Horford,3.5,9.6,11.9,0.432,1.59,1.12,150,0.432,74,76,1.880471,2.530353,1.176,0.88,50.5,7600,11400.0
27,Andrew Wiggins,1.7,7,15.8,0.353,0.88,1.19,204,0.353,68,136,1.50025,4.624,2.06375,0.81,42.0,7200,10800.0
24,Klay Thompson,2.4,4.3,19.8,0.399,0.75,1.38,266,0.399,143,123,3.566063,4.059,1.0,1,41.5,7400,11100.0
20,Marcus Smart,6.2,4.5,15.5,0.33,0.33,2.2,200,0.33,103,97,2.266,3.000533,2.733333,1.13,39.5,8000,12000.0
5,Draymond Green,6.3,6.9,8.7,0.261,1.13,2.75,102,0.261,23,79,0.375188,3.06125,1.437188,0.94,37.0,7000,10500.0
17,Jordan Poole,4.5,3.2,18.4,0.393,0.44,2.56,192,0.393,89,103,2.186063,4.184375,3.43875,0.94,35.5,6000,9000.0
12,Kevon Looney,2,7.7,6.1,0,0.31,0.56,66,0,0,66,0.0,2.751375,0.563,0.31,30.5,6800,10200.0
